In [ ]:
import pandas as pd
from pyprojroot import here

pd.set_option('display.max_rows', 100)

data = pd.read_csv(
    here() / 'data/raw/data.csv',
    index_col='datetime',
    parse_dates=True)

beds = data.loc[:,data.columns.str.startswith('Beds')]
data = data.drop(columns=beds.columns)

# Some hospitals don't have any data prior to 2018 so we drop these
drop_these = ['Beds:HC_10', 
              'Beds:HC_11_(Ward_01)',
              'Beds:HC_11_(Ward_02)',
              'Beds:HC_11_(Ward_03)',
              'Beds:UH_(Ward_01)',
              'Beds:UH_(Ward_02)',
              'Beds:HC_14',
              'Beds:RH_B_(Ward_01)',
              'Beds:RH_B_(Ward_02)'
              ]

beds = beds.drop(columns=drop_these)
conversion = {
    'Beds:HC_12' : 'Beds:HC_10',
    'Beds:HC_13' : 'Beds:HC_11',
    'Beds:HC_15' : 'Beds:HC_12'
}
beds = beds.rename(columns=conversion).copy()
beds = beds.apply(lambda row: row.fillna(row.mean()), axis=1)
data = pd.concat([data, beds], axis=1)

data = data[data.columns.sort_values()]

data.to_csv(here() / 'data/interim/data.csv')